## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-08-16-55-41 +0000,bbc,US skier Vonn crashes in downhill run while co...,https://www.bbc.com/sport/articles/c98q5g51z4n...
1,2026-02-08-16-53-38 +0000,nypost,"NYC cigar bar goes up in smoke, injuring 7",https://nypost.com/2026/02/08/us-news/nyc-ciga...
2,2026-02-08-16-52-15 +0000,bbc,Japan's governing party on course for landslid...,https://www.bbc.com/news/articles/cx2y7d2z29xo...
3,2026-02-08-16-47-10 +0000,nypost,"One dead, six hospitalized after tragedy strik...",https://nypost.com/2026/02/08/world-news/one-d...
4,2026-02-08-16-46-04 +0000,startribune,FBI concluded Jeffrey Epstein wasn't running a...,https://www.startribune.com/fbi-concluded-jeff...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2426/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
218,trump,18
20,election,11
83,super,10
84,bowl,10
15,japan,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
50,2026-02-08-12-00-00 +0000,wsj,The Trump economy in 2025 was a roller coaster...,https://www.wsj.com/economy/consumers/american...,48
123,2026-02-08-00-12-00 +0000,wsj,Whistleblower complaint against Tulsi Gabbard ...,https://www.wsj.com/politics/national-security...,47
55,2026-02-08-11-00-00 +0000,latimes,"Trump, California and the multi-front war over...",https://www.latimes.com/politics/story/2026-02...,43
61,2026-02-08-10-50-00 +0000,startribune,Analysis: Super Bowl GMs show just how off-tar...,https://www.startribune.com/vikings-general-ma...,43
148,2026-02-07-20-56-48 +0000,nypost,Epstein cultivated close ties with two key Rus...,https://nypost.com/2026/02/07/world-news/epste...,41


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
50,48,2026-02-08-12-00-00 +0000,wsj,The Trump economy in 2025 was a roller coaster...,https://www.wsj.com/economy/consumers/american...
61,43,2026-02-08-10-50-00 +0000,startribune,Analysis: Super Bowl GMs show just how off-tar...,https://www.startribune.com/vikings-general-ma...
16,40,2026-02-08-15-40-07 +0000,nyt,"Savannah Guthrie, in New Video, Promises to Pa...",https://www.nytimes.com/2026/02/07/us/savannah...
47,35,2026-02-08-12-03-53 +0000,nyt,"Heavy Snow Disrupts Japan Election, Forcing Po...",https://www.nytimes.com/2026/02/08/world/asia/...
148,32,2026-02-07-20-56-48 +0000,nypost,Epstein cultivated close ties with two key Rus...,https://nypost.com/2026/02/07/world-news/epste...
113,28,2026-02-08-01-01-15 +0000,nyt,Nithya Raman Announces She Will Run for Los An...,https://www.nytimes.com/2026/02/07/us/nithya-r...
150,24,2026-02-07-20-48-44 +0000,wapo,"At Winter Olympics, protest over cost and ICE ...",https://www.washingtonpost.com/sports/olympics...
123,22,2026-02-08-00-12-00 +0000,wsj,Whistleblower complaint against Tulsi Gabbard ...,https://www.wsj.com/politics/national-security...
76,21,2026-02-08-09-55-25 +0000,nypost,Suspect who allegedly shot top Russian general...,https://nypost.com/2026/02/08/world-news/suspe...
56,20,2026-02-08-11-00-00 +0000,latimes,Commentary: Fix the potholes or fight the powe...,https://www.latimes.com/politics/story/2026-02...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
